## Project 2: ETL
Extraction (E) and Transform (T) steps will be performed in this jupyter notebook.
Load (L) will be performed in PostGres.

In [1]:
# Dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

##     Step 1:  Extract Data

In [2]:
# csv file variable assignments
goodreads_path = "Resources/book_data.csv"
google_path = "Resources/google_books_1299.csv"
nyt_path = "Resources/books_uniq_weeks.csv"

# Import csv files as dataframes
goodreads_rdf = pd.read_csv(goodreads_path)
google_rdf = pd.read_csv(google_path, encoding="ANSI")
nyt_rdf = pd.read_csv(nyt_path, encoding="ISO-8859-1")

#goodreads_rdf.head()
#google_rdf.head()
nyt_rdf.head()

,publisher,dagger,book_review_link,author,primary_isbn10,price,primary_isbn13,sunday_review_link,date,first_chapter_link,contributor,title,age_group,weeks_on_list
0,Riverhead,0,https://www.nytimes.com/2015/01/05/books/the-g...,Paula Hawkins,1594634025,0,9.78159E+12,https://www.nytimes.com/2015/02/01/books/revie...,2/19/17,NaN,by Paula Hawkins,THE GIRL ON THE TRAIN,NaN,102
1,Scribner,0,NaN,Anthony Doerr,1501173219,0,9.7815E+12,https://www.nytimes.com/2014/05/11/books/revie...,5/7/17,NaN,by Anthony Doerr,ALL THE LIGHT WE CANNOT SEE,NaN,81
2,Vintage,0,NaN,E L James,525431888,0,9.78053E+12,NaN,3/5/17,NaN,by E. L. James,FIFTY SHADES DARKER,NaN,66
3,St. Martin's,0,NaN,Kristin Hannah,1466850604,0,9.78147E+12,NaN,10/29/17,NaN,by Kristin Hannah,THE NIGHTINGALE,NaN,63
4,Penguin Group,0,https://www.nytimes.com/2009/02/19/books/19mas...,Kathryn Stockett,1440697663,0,9.78144E+12,NaN,4/8/12,NaN,by Kathryn Stockett,THE HELP,NaN,58


## Step 2:  Transform - prep the data for Load (L) step.

In [3]:
# We discovered last minute (cry) that these Kraggle files had many duplicated ISBNS for books of different titles/authors

goodreads_rdf = goodreads_rdf.drop_duplicates(subset=['book_isbn'])
google_rdf = google_rdf.drop_duplicates(subset=['ISBN'])

In [4]:
# Convert ISBN to string

goodreads_rdf['book_isbn'] = goodreads_rdf['book_isbn'].astype("str")
#goodreads_rdf.dtypes
goodreads_rdf.head()
#google_rdf.dtypes

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9780440000000,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9780060000000,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9780680000000,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9780320000000,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
5,Markus Zusak,Trying to make sense of the horrors of World W...,First American Edition (US / CAN),Hardcover,9780380000000,552 pages,4.36,1485632,100821,The Book Thief,Historical|Historical Fiction|Fiction|Young Adult,https://images.gr-assets.com/books/1522157426l...


In [5]:
# Convert ISBN to string

google_rdf['ISBN'] = google_rdf['ISBN'].astype("str")
#google_rdf.dtypes
google_rdf.head()
#google_rdf.dtypes

,Unnamed: 0,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date
0,0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192.0,none,9781610000000,English,31-Jul-14
1,1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Barbara Allan,3.3,23,26.15,SAR,Determined to make a new start in her quaint h...,Kensington Publishing Corp.,288.0,"Fiction , Mystery &amp, Detective , Cozy , Gen...",9780760000000,English,1-Jul-07
2,2,The Art of Super Mario Odyssey,Nintendo,3.9,9,133.85,SAR,Take a globetrotting journey all over the worl...,Dark Horse Comics,368.0,"Games &amp, Activities , Video &amp, Electronic",9781510000000,English,5-Nov-19
3,3,Getting Away Is Deadly: An Ellie Avery Mystery,Sara Rosett,4.0,10,26.15,SAR,"With swollen feet and swelling belly, pregnant...",Kensington Publishing Corp.,320.0,none,9781620000000,English,1-Mar-09
4,4,"The Painted Man (The Demon Cycle, Book 1)",Peter V. Brett,4.5,577,28.54,SAR,The stunning debut fantasy novel from author P...,HarperCollins UK,544.0,"Fiction , Fantasy , Dark Fantasy",9780010000000,English,8-Jan-09


In [6]:
# Extract interested GOODREADS data fields into new dataframe
goodreads_df = goodreads_rdf[['book_isbn', 
                              'book_authors', 
                              'book_title', 
                              'book_desc', 
                              'book_edition',
                              'book_rating', 
                              'book_pages', 
                              'book_format', 
                              'genres', 
                              'image_url']]

# Rename headers as needed
goodreads_df.rename(columns={'book_isbn': 'isbn', 
                             'book_authors': 'gr_author', 
                             'book_title': 'gr_title', 
                             'book_desc': 'gr_desc', 
                             'book_edition': 'gr_edition',
                             'book_rating': 'goodreads_rating', 
                             'book_pages': 'gr_pages', 
                             'book_format': 'gr_format', 
                             'genres': 'gr_genres'}, inplace=True)



#goodreads_df.head()
#goodreads_df.count()
goodreads_df['isbn'].value_counts()

C:\Users\nvkar\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


9783870000000    1
9785950000000    1
9780910000000    1
789112000000     1
9781800000000    1
                ..
9789800000000    1
9780370000000    1
9783790000000    1
9781040000000    1
9788370000000    1
Name: isbn, Length: 549, dtype: int64

In [7]:
# Extract interested GOOGLE data fields into new dataframe
google_df = google_rdf[['ISBN', 
                        'author', 
                        'title', 
                        'rating', 
                        'price', 
                        'currency', 
                        'description', 
                        'publisher', 
                        'page_count', 
                        'generes', 
                        'published_date']]

# Rename headers as needed
google_df.rename(columns={'ISBN':'isbn', 
                             'author':'gg_author', 
                             'title':'gg_title', 
                             'rating':'gg_rating', 
                             'price':'gg_price', 
                             'currency':'gg_currency', 
                             'description':'gg_description', 
                             'publisher':'gg_publisher', 
                             'page_count':'gg_pages', 
                             'generes':'gg_genres', 
                             'published_date':'gg_published_date'}, inplace=True)

google_df.rename(columns={'ISBN':'isbn'}, inplace=True)


#google_df.dtypes
#google_df.head()
#google_df.count()
google_df['isbn'].value_counts()

9780680000000    1
9780550000000    1
9781600000000    1
9780140000000    1
9781400000000    1
                ..
9781740000000    1
9781520000000    1
9780070000000    1
9781430000000    1
9780430000000    1
Name: isbn, Length: 66, dtype: int64

In [8]:
# Merge google and goodreads data to form the beginnings of the book table
books_df = pd.merge(goodreads_df, google_df, on="isbn", how="inner")
books_df['isbn'].value_counts()

9780400000000    1
9781470000000    1
9780140000000    1
9781640000000    1
9781400000000    1
                ..
9781520000000    1
9780070000000    1
9781430000000    1
9780360000000    1
9780430000000    1
Name: isbn, Length: 62, dtype: int64

In [9]:
books_df.count()

isbn                 62
gr_author            62
gr_title             62
gr_desc              61
gr_edition           18
goodreads_rating     62
gr_pages             61
gr_format            61
gr_genres            59
image_url            61
gg_author            62
gg_title             62
gg_rating            59
gg_price             62
gg_currency          62
gg_description       62
gg_publisher         62
gg_pages             62
gg_genres            62
gg_published_date    62
dtype: int64

In [10]:
books_df.dtypes

isbn                  object
gr_author             object
gr_title              object
gr_desc               object
gr_edition            object
goodreads_rating     float64
gr_pages              object
gr_format             object
gr_genres             object
image_url             object
gg_author             object
gg_title              object
gg_rating            float64
gg_price             float64
gg_currency           object
gg_description        object
gg_publisher          object
gg_pages             float64
gg_genres             object
gg_published_date     object
dtype: object

In [11]:
books_df.head()

,isbn,gr_author,gr_title,gr_desc,gr_edition,goodreads_rating,gr_pages,gr_format,gr_genres,image_url,gg_author,gg_title,gg_rating,gg_price,gg_currency,gg_description,gg_publisher,gg_pages,gg_genres,gg_published_date
0,9780060000000,Harper Lee,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,50th Anniversary,4.27,324 pages,Paperback,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...,Jason Schreier,"Blood, Sweat, and Pixels: The Triumphant, Turb...",4.3,50.25,SAR,NATIONAL BESTSELLERDeveloping video gamesâ€”he...,HarperCollins,304.0,"Games &amp, Activities , Video &amp, Electronic",5-Sep-17
1,9780680000000,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,Pride and Prejudice,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",4.25,279 pages,Paperback,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...,Charles Duhigg,The Power of Habit: Why We Do What We Do in Li...,4.1,62.10,SAR,NEW YORK TIMESÂ BESTSELLER â€¢ This instant cl...,Random House,416.0,"Business &amp, Economics , Organizational Beha...",28-Feb-12
2,9780070000000,C.S. Lewis|Pauline Baynes,The Chronicles of Narnia,"Journeys to the end of the world, fantastic cr...",Reissue Edition,4.25,767 pages,Paperback,Fantasy|Classics|Fiction|Young Adult|Childrens,https://images.gr-assets.com/books/1449868701l...,David Ulrich,HR from the Outside In: Six Competencies for t...,3.6,133.90,SAR,â€œThis definitive work on HR competencies pro...,McGraw Hill Professional,272.0,"Business &amp, Economics , Training",20-Jul-12
3,9780450000000,George Orwell,Animal Farm,مزرعة الحيوانات هي رائعة جورج أورويل الخالدة.....,NaN,3.90,122 pages,Paperback,Classics|Fiction|Science Fiction|Dystopia|Fant...,https://images.gr-assets.com/books/1424037542l...,Madeleine Roux,Salvaged,4.0,47.78,SAR,A WOMAN ON THE RUN. A CAPTAIN ADRIFT IN SPACE....,Penguin,368.0,"Fiction , Science Fiction , Action &amp, Adven...",15-Oct-19
4,9780350000000,J.R.R. Tolkien,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,لجزء الثالث من ملحمة جيه أر أر تولكين الرائعة ...,Hobbit Movie Tie-in Boxed set,4.59,1728 pages,Mass Market Paperback,Fantasy|Fiction|Classics,https://images.gr-assets.com/books/1346072396l...,Pierce Brown,Morning Star,4.8,62.10,SAR,#1 NEW YORK TIMES BESTSELLER â€¢Â Red Rising t...,Del Rey,544.0,"Fiction , Science Fiction , Action &amp, Adven...",9-Feb-16


In [12]:
# Combine key fields into new column based on conditional logic

books_df['title'] = np.where(books_df['gg_title'] == "", books_df['gr_author'],books_df['gg_title'])
books_df['author'] = np.where(books_df['gg_author'] == "", books_df['gr_author'],books_df['gg_author'])

books_df.count()

isbn                 62
gr_author            62
gr_title             62
gr_desc              61
gr_edition           18
goodreads_rating     62
gr_pages             61
gr_format            61
gr_genres            59
image_url            61
gg_author            62
gg_title             62
gg_rating            59
gg_price             62
gg_currency          62
gg_description       62
gg_publisher         62
gg_pages             62
gg_genres            62
gg_published_date    62
title                62
author               62
dtype: int64

In [27]:
final_books_df = books_df[['isbn','title', 'author', 'gg_publisher', 'gg_published_date']]
final_books_df = final_books_df.rename(columns={'gg_publisher':'publisher', 
                             'gg_published_date':'publication_date'}) 
final_books_df


,isbn,title,author,publisher,publication_date
0,9780060000000,"Blood, Sweat, and Pixels: The Triumphant, Turb...",Jason Schreier,HarperCollins,5-Sep-17
1,9780680000000,The Power of Habit: Why We Do What We Do in Li...,Charles Duhigg,Random House,28-Feb-12
2,9780070000000,HR from the Outside In: Six Competencies for t...,David Ulrich,McGraw Hill Professional,20-Jul-12
3,9780450000000,Salvaged,Madeleine Roux,Penguin,15-Oct-19
4,9780350000000,Morning Star,Pierce Brown,Del Rey,9-Feb-16
...,...,...,...,...,...
57,9783640000000,"Business Process Management: Concepts, Languag...",Mathias Weske,Springer Science & Business Media,3-May-12
58,9781260000000,Tintinalli's Emergency Medicine: A Comprehensi...,Judith E. Tintinalli,McGraw Hill Professional,22-Oct-19
59,9788070000000,The Art of War (The Classic Lionel Giles Trans...,Sun Tzu,e-artnow,2-Jun-12
60,9783740000000,15 Delicious Slow Cooker Recipes,Sallie Stone,BookRix,28-Oct-19


In [28]:
# This is the final_books_df to load
final_books_df.head()

,isbn,title,author,publisher,publication_date
0,9780060000000,"Blood, Sweat, and Pixels: The Triumphant, Turb...",Jason Schreier,HarperCollins,5-Sep-17
1,9780680000000,The Power of Habit: Why We Do What We Do in Li...,Charles Duhigg,Random House,28-Feb-12
2,9780070000000,HR from the Outside In: Six Competencies for t...,David Ulrich,McGraw Hill Professional,20-Jul-12
3,9780450000000,Salvaged,Madeleine Roux,Penguin,15-Oct-19
4,9780350000000,Morning Star,Pierce Brown,Del Rey,9-Feb-16


In [29]:
# Prepping nyt data table

nyt_df = nyt_rdf[['primary_isbn13','weeks_on_list']]

#nyt_df = books_df[['goodreads_rating', 'gg_rating']]
nyt_df = nyt_df.rename(columns={'primary_isbn13':'isbn', 
                              'weeks_on_list':'weeks'}) 
# Drop any duplicate isbns
nyt_df = nyt_df.drop_duplicates(subset=['isbn'])  

# nyt_df['isbn'].count()
nyt_df
                          

,isbn,weeks
0,9.78159E+12,102
1,9.7815E+12,81
2,9.78053E+12,66
3,9.78147E+12,63
4,9.78144E+12,58
...,...,...
2161,9.78079E+12,0
2173,A00B005C50KA8,0
2178,A00B00547KH66,0
2199,A00B0052YFNFQ,0


In [30]:
# Prepping Ratings table

ratings_df = books_df[['isbn', 'goodreads_rating', 'gg_rating']]
ratings_df = ratings_df.rename(columns={'goodreads_rating':'goodreads_rtg', 
                              'gg_rating':'google_rtg'}) 
ratings_df

,isbn,goodreads_rtg,google_rtg
0,9780060000000,4.27,4.3
1,9780680000000,4.25,4.1
2,9780070000000,4.25,3.6
3,9780450000000,3.90,4.0
4,9780350000000,4.59,4.8
...,...,...,...
57,9783640000000,4.80,2.5
58,9781260000000,4.50,NaN
59,9788070000000,3.77,4.6
60,9783740000000,5.00,5.0


In [31]:
final_ratings_df = pd.merge(ratings_df, nyt_df, on="isbn", how="left")

final_ratings_df["nyt_ind"] = np.where(final_ratings_df['weeks'] >= 1, True, False)

final_ratings_df = final_ratings_df[['isbn', 'google_rtg', 'goodreads_rtg', 'nyt_ind', 'weeks']]

final_ratings_df.head()

,isbn,google_rtg,goodreads_rtg,nyt_ind,weeks
0,9780060000000,4.3,4.27,False,NaN
1,9780680000000,4.1,4.25,False,NaN
2,9780070000000,3.6,4.25,False,NaN
3,9780450000000,4.0,3.90,False,NaN
4,9780350000000,4.8,4.59,False,NaN


In [32]:
final_ratings_df.dtypes

isbn              object
google_rtg       float64
goodreads_rtg    float64
nyt_ind             bool
weeks            float64
dtype: object

## Create database connection

In [36]:
connection_string = "postgres:postgres@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [37]:
 # Confirm tables
engine.table_names()

['books', 'ratings']

In [38]:
final_books_df.to_sql(name='books', con=engine, if_exists='append', index=False)

In [39]:
final_ratings_df.to_sql(name='ratings', con=engine, if_exists='append', index=False)